In [93]:
import openpyxl as xl
import requests 
from bs4 import BeautifulSoup

In [94]:
class Player:    
    def __init__(self, name, link, team, position):
        self.name = name
        self.link = link
        self.team = team
        self.position = position
    
    def getName(self):
        return self.name

    def getLink(self):
        return self.link
    
    def getTeam(self):
        return self.team
    
    def getPosition(self):
        return self.position

In [95]:
def comLinkList(link):

    playersList = []

    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')

    linkList = []

    x = soup.find(id='comp') 
    a = x['action']
    print(a)
    y = x.find('input')
    b = y['value']
    print(b)
    z = x.find_all('option')

    valueList = []

    i = 0
    for m in z:
        if (i != 0):        
            c = m['value']
            valueList.append(c)
        i = i + 1

    for v in valueList:
        link = a + '?comp_id=' +  str(b) + '&club_id=' + str(v)
        linkList.append(link)

    return linkList

In [96]:
# player scrape from starbunker 6 nation team function

def scrapePlayers(linkX,team):
    
    playersList = []

    page = requests.get(linkX)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    playerClass = soup.find(class_='hozList')    
    players = playerClass.find_all('a')
    
    for p in players:
        x = p.find_previous("ul")
        y = x.find_previous_sibling('h3').string
        playersList.append(Player(p.find('p').string, p['href'], team, y))
        
    return playersList

In [97]:
# list for scrapped players from web

scrappedPlayerList = []

scrappedPlayerListEng = []
scrappedPlayerListFra = []
scrappedPlayerListIre = []
scrappedPlayerListIta = []
scrappedPlayerListSco = []
scrappedPlayerListWal = []

In [98]:
# players adding to scarppedPlayerList function

def playersAddToSPL(playerList, scrapeList):
    
    for sp in scrapeList:
        for p in playerList:
            if(sp.getName == p.getName):
                playerList.remove(p)
                
    for p in playerList:
        scrapeList.append(p)

In [99]:
#  adding players to scarppedPlayerList21

def teamAllPlayers(link, team, scrappedPlayerList):
    for l in comLinkList(link):
        playersAddToSPL(scrapePlayers(l, team), scrappedPlayerList)


In [100]:
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=116', 'England', scrappedPlayerListEng)
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=120', 'France', scrappedPlayerListFra)
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=119', 'Ireland', scrappedPlayerListIre)
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=122', 'Italy', scrappedPlayerListIta)
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=118', 'Scotland', scrappedPlayerListSco)
teamAllPlayers('https://rugby.statbunker.com/competitions/getCompClubSquad?comp_id=653&club_id=117', 'Wales', scrappedPlayerListWal)

https://rugby.statbunker.com/competitions/getCompClubSquad
116
https://rugby.statbunker.com/competitions/getCompClubSquad
120
https://rugby.statbunker.com/competitions/getCompClubSquad
119
https://rugby.statbunker.com/competitions/getCompClubSquad
122
https://rugby.statbunker.com/competitions/getCompClubSquad
118
https://rugby.statbunker.com/competitions/getCompClubSquad
117


In [101]:
def addToML(mainList, compList):
    
    list1 = mainList
    list2 = compList
    
    for player in list2:
        list1.append(player)
        
    mainList = list1

In [102]:
# adding to scrappedPlayerList list

addToML(scrappedPlayerList, scrappedPlayerListEng)
addToML(scrappedPlayerList, scrappedPlayerListFra)
addToML(scrappedPlayerList, scrappedPlayerListIre)
addToML(scrappedPlayerList, scrappedPlayerListIta)
addToML(scrappedPlayerList, scrappedPlayerListSco)
addToML(scrappedPlayerList, scrappedPlayerListWal)

print(len(scrappedPlayerList))

1701


In [103]:
# loading Players xl sheet

wb = xl.load_workbook('Kaggle.xlsx')
ws1 = wb['Player Details']
ws2 = wb['Prediction Stats']

In [104]:
col_headers = ["Name",
               "Team",
               "Position",
               "Matches Played", 
               "Matches Selected",
               "Matches Started",
               "Matches as a Sub",
               "Captain", 
               "Minutes Played", 
               "Came On", 
               "Minutes as a Sub",
               "Temporary Sub", 
               "Came Off",
               "Tries",
               "Drop Goals",
               "Conversion Attempts", 
               "Conversion Scored",
               "Penalties Attempts",
               "Penalties Scored", 
               "Points",
               "Sin Bin",
               "Rescinded", 
               "Red Card"]

index = 0
for col in range(1, len(col_headers) + 1):
    ws2.cell(1, col).value = col_headers[index]
    index = index + 1

wb.save('Kaggle.xlsx')

In [105]:
# scrape player stats function
def addPlayerDetails(player, row):
    
    titleList = []
    statList = []

    page = requests.get(player.getLink())
    soup = BeautifulSoup(page.content, 'html.parser')
    
    menu = soup.find(class_='navMain')
    titles = menu.find_all('a')
    
    for title in titles:
        titleList.append(title['href'])
    
    statList = allTimeStats(titleList[3], player.getTeam())
    
    rounds = len(statList)//20
    
    index = 0
    
    for rod in range (0, rounds):
        ws2.cell(row, 1).value = player.getName()
        ws2.cell(row, 2).value = player.getTeam()
        ws2.cell(row, 3).value = player.getPosition()
        for col in range(4, 24):
            ws2.cell(row, col).value = statList[index]
            index = index + 1
        for ws1_row in range(2, ws1.max_row + 1):
            if (ws1.cell(ws1_row, 1).value == ws2.cell(row, 1).value):
                if (ws1.cell(ws1_row, 12).value == ws2.cell(row, 2).value):
                    if (ws1.cell(ws1_row, 13).value == ws2.cell(row, 4).value):
                        row = row - 1
        row = row + 1
                   
    return row

In [106]:
# scrape all time player stats function

def allTimeStats(link, team):
    statList = []

    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    statValues = soup.find_all('td')

    for statValue in statValues:
        if (str(statValue.string).isdigit()):
            statList.append(int(statValue.string))
            
    return statList

In [ ]:
# adding some player stats scrapped from starbunker to Players xl sheet

row = 2

for player in scrappedPlayerList:
    row = addPlayerDetails(player, row)
    wb.save('Kaggle.xlsx')

print("Done")